In [254]:
import pandas as pd
from playwright.async_api import async_playwright
from tqdm.notebook import tqdm
import time
import pdb
import csv
from PIL import Image
import requests
import requests
from bs4 import BeautifulSoup
import pdb
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [99]:
list1 = [0.45, 1.5, 2, 1, 1.75, 2.5, 2.10]

time_input = int(random.choice(list1))
time_input

2

In [518]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

In [323]:
state_code = list(range(13,14))
district_code = list(range(1,76))

In [324]:
len(state_code)

1

In [325]:
len(district_code)

75

In [329]:
state_code = list(range(13,14))
district_code = list(range(1,76))
def making_url_list(state_code, district_code):
    list_of_urls = []
    for each_state in state_code:
        input1 = each_state
        for each_district in district_code:
            input2 = each_district
            url_stitch = "https://justiceclock.ecourts.gov.in/justiceClock/?p=home/district&fstate_code=" 
            state_code = f"{input1}"
            district_stitch = "&fdist_code=" 
            district_code = f"{input2}"
            final_url = url_stitch + state_code + district_stitch + district_code
            list_of_urls.append(final_url)
    return list_of_urls

In [330]:
urls = making_url_list(state_code, district_code)

In [513]:
len(urls)

75

In [374]:
urls[8]

'https://justiceclock.ecourts.gov.in/justiceClock/?p=home/district&fstate_code=13&fdist_code=9'

In [505]:
# list_of_lists = []

In [543]:
async def simpler_justice_clock(urls):
    for url in urls:
        await page.goto(url)
        time.sleep(5)
        try:
            await page.locator('#toggleBtn').click()
        except:
            div_element = page.locator("div")
#             for div_element in div_elements:
            inner_text = div_element.inner_text()
        # Check if the inner text matches the target content
            if inner_text == "Access Denied...!!!":
                print(inner_text)
                continue
        try:
            await page.wait_for_selector(".table")
        except:
            continue
        time.sleep(time_input)
        table_text = await page.locator(".table").inner_text()
        time.sleep(time_input)
        age = table_text.split()
        time.sleep(time_input)
        state_text = await page.locator("#sp_state").inner_text()
        if len(state_text) > 0:
            state_name = state_text.split("OF ")[1].title()
            age.append(state_name)
        else:
            continue
        time.sleep(time_input)        
        district_text = await page.locator('#fest_code option').all_text_contents()  
        time.sleep(time_input)
        district_name = district_text[0].split(",")[0]
        district = district_name.split("-")[1].title()
        age.append(district)
        print("it has reached here")
        if age in list_of_lists:
            print("district done")
            continue
        else:
            list_of_lists.append(age)
    return

In [546]:
await simpler_justice_clock(urls)

it has reached here
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
it has reached here
district done
it has reached here
district done
it has reached here
district done
it has reached here
it has reached here
district done
it has reached here
it has reached here
district done
it has reached h

In [547]:
len(list_of_lists)

90

In [598]:
list_of_dictionaries = []

In [599]:
for age1 in list_of_lists:
    age_dict = {}
    age_dict['state'] = age1[30]
    age_dict['district'] = age1[31]
    age_dict[age1[5]] = age1[6]
    age_dict[age1[8]] = age1[9]
    age_dict[age1[11]] = age1[12]
    age_dict[age1[14]] = age1[15]
    age_dict[age1[17]] = age1[18]
    age_dict[age1[20]] = age1[21]
    age_dict[age1[23]+age1[24]] = age1[25]
    age_dict[age1[27]] = age1[28]
    list_of_dictionaries.append(age_dict)

In [600]:
df = pd.DataFrame(list_of_dictionaries)
df.head()

,state,district,0-1,2-3,4-5,6-10,11-20,21-30,Above30,Total,...,74,72%,4425,65%,174,4046,105%,33,17%,1769
0,Uttar Pradesh,Allahabad,51417,61473,50023,82823,73661,15851,1899,337147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85274,78306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Uttar Pradesh,Gorakhpur,63402,58408,28055,63964,45401,10051,1287,270568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Uttar Pradesh,Hardoi,52294,26377,27983,27454,25526,1385,227,161246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Uttar Pradesh,Chitrakoot,10422,8034,6545,5057,1472,48,7,31585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [601]:
df[df['district'].str.isalpha()].reset_index(drop=True).to_csv("rough_justice_clock_scrape.csv", index=False)

In [603]:
df_auto = pd.read_csv("rough_justice_clock_scrape.csv", usecols=["0-1","2-3","4-5","6-10","11-20", "21-30", "Above30","Total", "state", "district"])
df_auto.to_csv("masters_project/justice_clock_for_autoupdating.csv", index=False)